# Part I - Loan Data From Prosper
## by Roseland Ambuku

## Introduction
The 113,937 loans in this data set have an average of 81 variables, such as loan amount, borrower rate (or interest rate),current loan status, borrower income, and a lot more.

## Preliminary Wrangling


In [2]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [3]:
df_loandata = pd.read_csv('prosperLoanData.csv')

In [4]:
df_loandata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 81 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   ListingKey                           113937 non-null  object 
 1   ListingNumber                        113937 non-null  int64  
 2   ListingCreationDate                  113937 non-null  object 
 3   CreditGrade                          28953 non-null   object 
 4   Term                                 113937 non-null  int64  
 5   LoanStatus                           113937 non-null  object 
 6   ClosedDate                           55089 non-null   object 
 7   BorrowerAPR                          113912 non-null  float64
 8   BorrowerRate                         113937 non-null  float64
 9   LenderYield                          113937 non-null  float64
 10  EstimatedEffectiveYield              84853 non-null   float64
 11  EstimatedLoss

In [5]:
df_loandata.shape

(113937, 81)

In [6]:
print(df_loandata.dtypes)

ListingKey                      object
ListingNumber                    int64
ListingCreationDate             object
CreditGrade                     object
Term                             int64
                                ...   
PercentFunded                  float64
Recommendations                  int64
InvestmentFromFriendsCount       int64
InvestmentFromFriendsAmount    float64
Investors                        int64
Length: 81, dtype: object


In [7]:
df_loandata.head(10)

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20
5,0F05359734824199381F61D,1074836,2013-12-14 08:26:37.093000000,NaN,60,Current,NaN,0.15425,0.1314,0.1214,...,-25.33,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
6,0F0A3576754255009D63151,750899,2013-04-12 09:52:56.147000000,NaN,36,Current,NaN,0.31032,0.2712,0.2612,...,-22.95,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
7,0F1035772717087366F9EA7,768193,2013-05-05 06:49:27.493000000,NaN,36,Current,NaN,0.23939,0.2019,0.1919,...,-69.21,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
8,0F043596202561788EA13D5,1023355,2013-12-02 10:43:39.117000000,NaN,36,Current,NaN,0.07620,0.0629,0.0529,...,-16.77,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
9,0F043596202561788EA13D5,1023355,2013-12-02 10:43:39.117000000,NaN,36,Current,NaN,0.07620,0.0629,0.0529,...,-16.77,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1


### Data Cleaning
##### The following are some of the steps that shall be carried out:
1. Choosing the subset of features(variables) that are important in the study. 
2. Dropping all duplicated rows on the basis of Listing Number.
3. Converting the following datatypes: ListingCreationDate to datetime, TotalInquires and TotalTrades to int.
4. Fill in the missing values of occupation and DebtToIncomeRatio.
5. Removing rows that do not have ProsperRating.
6. Changing Listing category (numeric) to string

In [9]:
#Creating a subset of the entire dataframe to select the features we are interested in
clmns = ['ListingNumber','ListingCreationDate','LoanOriginalAmount','LoanStatus','ListingCategory (numeric)','BorrowerState','BorrowerAPR','BorrowerRate','StatedMonthlyIncome', 'ProsperRating (Alpha)', 
        'Occupation','Term','EmploymentStatus','TotalInquiries','DebtToIncomeRatio','MonthlyLoanPayment','TotalTrades','Investors']
df_proloans = df_loandata[clmns]

In [10]:
#Information on the subset of the dataframe
df_proloans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113937 entries, 0 to 113936
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ListingNumber              113937 non-null  int64  
 1   ListingCreationDate        113937 non-null  object 
 2   LoanOriginalAmount         113937 non-null  int64  
 3   LoanStatus                 113937 non-null  object 
 4   ListingCategory (numeric)  113937 non-null  int64  
 5   BorrowerState              108422 non-null  object 
 6   BorrowerAPR                113912 non-null  float64
 7   BorrowerRate               113937 non-null  float64
 8   StatedMonthlyIncome        113937 non-null  float64
 9   ProsperRating (Alpha)      84853 non-null   object 
 10  Occupation                 110349 non-null  object 
 11  Term                       113937 non-null  int64  
 12  EmploymentStatus           111682 non-null  object 
 13  TotalInquiries             11

In [11]:
#Looking at rows to see errors present
df_proloans.sample(20)

,ListingNumber,ListingCreationDate,LoanOriginalAmount,LoanStatus,ListingCategory (numeric),BorrowerState,BorrowerAPR,BorrowerRate,StatedMonthlyIncome,ProsperRating (Alpha),Occupation,Term,EmploymentStatus,TotalInquiries,DebtToIncomeRatio,MonthlyLoanPayment,TotalTrades,Investors
106383,634954,2012-09-06 03:39:16.020000000,4000,Current,1,MA,0.29464,0.2684,3994.583333,D,Clerical,60,Full-time,7.0,0.35,121.76,18.0,7
33476,1080957,2013-12-12 08:41:53.577000000,4284,Current,2,KS,0.16304,0.1400,10250.000000,B,Executive,60,Employed,4.0,0.06,99.68,27.0,3
15116,644726,2012-09-25 05:22:42.997000000,15000,Current,1,MI,0.30697,0.2804,5833.333333,D,Tradesman - Mechanic,60,Employed,11.0,0.12,467.40,29.0,16
30262,527997,2011-09-18 22:31:30.547000000,4000,Current,1,CA,0.35643,0.3199,2000.000000,HR,Other,36,Employed,5.0,0.56,174.20,11.0,60
86295,537113,2011-11-03 12:33:57.347000000,4000,Current,1,MA,0.35797,0.3177,26875.000000,HR,Professional,36,Employed,5.0,NaN,173.71,24.0,13
101569,538604,2011-11-11 12:12:16.317000000,2500,Completed,7,AZ,0.19088,0.1619,3230.916667,B,Administrative Assistant,36,Employed,4.0,0.30,88.13,38.0,43
19657,1069939,2013-12-31 11:00:25.577000000,6200,Current,1,TN,0.18990,0.1535,3500.000000,B,Social Worker,36,Employed,9.0,0.32,215.99,44.0,1
97121,332116,2008-05-13 18:36:54.693000000,10000,Completed,1,GA,0.16688,0.1525,7083.333333,NaN,Professional,36,Part-time,8.0,0.25,347.88,43.0,288
38813,498849,2011-03-22 08:35:15.853000000,4500,Completed,3,TN,0.29510,0.2599,4125.000000,D,Psychologist,36,Employed,1.0,0.37,181.28,42.0,56
63763,35689,2006-08-31 10:27:30.303000000,15000,Completed,0,NaN,0.29776,0.2900,3750.000000,NaN,Other,36,Not available,20.0,0.36,628.59,NaN,93


In [46]:
# Dropping any duplicate values in the dataset
df_proloans = df_proloans.drop_duplicates()

In [34]:
#Selecting the rows in ProsperRating (Alpha) that are not null and where missing values can not be filled
df_proloans = df_proloans[df_proloans['ProsperRating (Alpha)'].notnull()]

In [47]:
# Flling in the missing values of DebttoIncome using the mean value of the column
df_proloans.DebtToIncomeRatio = df_proloans.DebtToIncomeRatio.fillna(df_proloans.DebtToIncomeRatio.mean())

In [48]:
#Filling in the missing values in Occupation with unknown
df_proloans.Occupation = df_proloans.Occupation.fillna('Unknown')

In [72]:
#Converting the datatypes of the necessary columns
df_proloans.TotalInquiries = df_proloans.TotalInquiries.astype('Int64')
df_proloans.TotalTrades = df_proloans.TotalTrades.astype('Int64')

In [82]:
# Splitting ListingCreationDate column into year,month,day,time
df_proloans['Year']=df_proloans['ListingCreationDate'].apply(lambda x: x.split("-")[0]).astype(str)
df_proloans['Month'] = df_proloans['ListingCreationDate'].apply(lambda x: x.split("-")[1]).astype(str)

KeyError: 'ListingCreationDate'

In [52]:
# Changing the numerical values of months with the actual names
df_proloans.month.replace(['01','02','03','04','05','06','07','08','09','10','11','12'],['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec'],inplace=True)
df_proloans.month.unique()

array(['Aug', 'Feb', 'Jan', 'Oct', 'Sept', 'Dec', 'Apr', 'May', 'Jul',
       'Nov', 'Jun', 'Mar'], dtype=object)

In [83]:
df_proloans['ListingCreationDate'] = df_proloans['ListingCreationDate'].apply(lambda x: x.split("-")[2]).astype(str)

KeyError: 'ListingCreationDate'

In [84]:
df_proloans['Day'] = df_proloans['ListingCreationDate'].apply(lambda x: x.split(" ")[0]).astype(str)
df_proloans['Time'] = df_proloans['ListingCreationDate'].apply(lambda x: x.split(" ")[1]).astype(str)

KeyError: 'ListingCreationDate'

In [85]:
#Dropping ListingCreationDate column in the original dataset
df_proloans.drop(columns=['ListingCreationDate'],inplace=True)

KeyError: "['ListingCreationDate'] not found in axis"

### Exploratory Data Analysis

In [86]:
df_proloans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113066 entries, 0 to 113936
Data columns (total 21 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ListingNumber              113066 non-null  int64  
 1   LoanOriginalAmount         113066 non-null  int64  
 2   LoanStatus                 113066 non-null  object 
 3   ListingCategory (numeric)  113066 non-null  int64  
 4   BorrowerState              107551 non-null  object 
 5   BorrowerAPR                113041 non-null  float64
 6   BorrowerRate               113066 non-null  float64
 7   StatedMonthlyIncome        113066 non-null  float64
 8   ProsperRating (Alpha)      83982 non-null   object 
 9   Occupation                 113066 non-null  object 
 10  Term                       113066 non-null  int64  
 11  EmploymentStatus           110811 non-null  object 
 12  TotalInquiries             111907 non-null  Int64  
 13  DebtToIncomeRatio          11

In [24]:
# Descriptive Statistics of the numerical variables
df_loandata.describe()

,ListingNumber,Term,BorrowerAPR,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperScore,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
count,8.485300e+04,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,...,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000,84853.000000
mean,7.738749e+05,42.486135,0.226658,0.196022,0.186017,0.168661,0.080306,0.096068,4.072243,5.950067,...,-54.642104,-8.206068,376.088798,367.724881,7.647064,0.998098,0.017560,0.008049,4.315558,68.264669
std,2.363387e+05,11.640346,0.079916,0.074631,0.074631,0.068467,0.046764,0.030403,1.673227,2.376501,...,59.729078,75.185535,1720.769115,1700.211129,119.165278,0.020742,0.187963,0.109721,117.005989,95.195831
min,4.162750e+05,12.000000,0.045830,0.040000,0.030000,-0.182700,0.004900,-0.182700,1.000000,1.000000,...,-589.950000,-4865.080000,-94.200000,-504.410000,0.000000,0.700000,0.000000,0.000000,0.000000,1.000000
25%,5.580860e+05,36.000000,0.163280,0.135900,0.125900,0.115670,0.042400,0.074080,3.000000,4.000000,...,-72.300000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
50%,7.392460e+05,36.000000,0.219450,0.187500,0.177500,0.161500,0.072400,0.091700,4.000000,6.000000,...,-34.900000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,32.000000
75%,9.794360e+05,60.000000,0.292540,0.257400,0.247400,0.224300,0.112000,0.116600,5.000000,8.000000,...,-14.280000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,97.000000
max,1.255725e+06,60.000000,0.423950,0.360000,0.340000,0.319900,0.366000,0.283700,7.000000,11.000000,...,3.010000,0.000000,25000.000000,25000.000000,7780.030000,1.012500,19.000000,9.000000,11000.000000,1189.000000


### What is the structure of your dataset?

There are 21 features and 113,066 loans. Some of the features included are StatedMonthlyIncome, Investors, BorrowerAPR, LoanStatus among others.

### What is/are the main feature(s) of interest in your dataset?

The main features of interest are 'DebtToIncome Ratio','Employment Status', they are used to figure out the loan outcome status of loan applications by clients of Prosper.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?

"Monthly Income","Loan Term","Prosper Rating","Occupation" features in the dataset will assist in supporting the investigation into the features of interest.

## Univariate Exploration


### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?

> Your answer here!

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?

> Your answer here!

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

## Conclusions
>You can write a summary of the main findings and reflect on the steps taken during the data exploration.



> Remove all Tips mentioned above, before you convert this notebook to PDF/HTML


> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML or PDF` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!

